# Постановка задачи

1. Определить выживаемость пассажиров "Титаника" с помощью нейронной сети (https://www.kaggle.com/c/titanic/data).
2. Сравнить результат с полученным в предыдущем решении этой задачи (titanic.ipynb).

In [145]:
import numpy as np # импортируем библиотеку Numpy
import pandas as pd # импортируем библиотеку Pandas
import matplotlib.pyplot as plt # импортируем библиотеку Matplotlib

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import np_utils, plot_model

In [146]:
titanic_train_data = pd.read_csv('https://raw.githubusercontent.com/Andruchelli/Titanic/main/train.csv') # прочитаем файл с тренировочными данными
titanic_test_data = pd.read_csv('https://raw.githubusercontent.com/Andruchelli/Titanic/main/test.csv') # прочитаем файл с тестовыми данными, который будет использоваться для решения задачи по определению выживаемости пассажиров

In [147]:
titanic_train_data.head() # проверяем, что первый файл прочитался

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [148]:
titanic_train_data.shape # смотрим размерность датасета с тренировочными данными

(891, 12)

In [149]:
titanic_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# Предобработка данных

In [150]:
df = titanic_train_data[['Survived', 'Age', 'Pclass']] # создаём новый датасет для определения выживаемости по двум параметрам: Age и Pclass
df.head()

,Survived,Age,Pclass
0,0,22.0,3
1,1,38.0,1
2,1,26.0,3
3,1,35.0,1
4,0,35.0,3


In [151]:
df.shape # проверяем размерность нового датасета

(891, 3)

In [152]:
np.sum(df['Survived'].isna()) # считаем количество ячеек в столбце Survived со значением NaN

0

In [153]:
np.sum(df['Pclass'].isna()) # считаем количество ячеек в столбце Pclass со значением NaN

0

In [154]:
np.sum(df['Age'].isna()) # считаем количество ячеек в столбце Age со значением NaN

177

Для дальнейших исследований стоит почистить наш новый датафрейм, так как в столбце Age мы получили 177 строк со значением NaN, которые не подходят для обучения модели.

In [155]:
df_dropna = df.dropna(axis=0) # избавляемся от всех рядов, где имеются значения NaN, с помощью метода dropna; axis=0 означает, что мы сбрасываем ряды, где содержатся значения NaN
df_dropna.shape # проверяем размерность нового датасета

(714, 3)

In [156]:
df_dropna.head()

,Survived,Age,Pclass
0,0,22.0,3
1,1,38.0,1
2,1,26.0,3
3,1,35.0,1
4,0,35.0,3


In [157]:
titanic_test_data.head() # проверяем, что второй файл, содержащий тестовые данные, прочитался

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [158]:
np.sum(titanic_test_data['Pclass'].isna()) # считаем количество ячеек в столбце Pclass со значением NaN

0

In [159]:
np.sum(titanic_test_data['Age'].isna()) # считаем количество ячеек в столбце Age со значением NaN

86

In [160]:
titanic_test_dropna = df.dropna(axis=0) # избавляемся от всех рядов, где имеются значения NaN, с помощью метода dropna; axis=0 означает, что мы сбрасываем ряды, где содержатся значения NaN
df_dropna.shape # проверяем размерность нового датасета

(714, 3)

In [161]:
X_train = df_dropna.drop('Survived', axis=1) # задаём в тренировочные данные для обучения модели столбцы Age и Pclass из тренировочного датасета
Y_train = df_dropna['Survived'] # задаём в полученный результат для тренировочных данных столбец Survived из тренировочного датасета

X_test = titanic_test_dropna[['Age', 'Pclass']] # задаём в тестовые данные, по которым будет проводиться обучение модели, столбцы Age и Pclass из тестоового датасета 

In [162]:
X_train.head()

,Age,Pclass
0,22.0,3
1,38.0,1
2,26.0,3
3,35.0,1
4,35.0,3


In [163]:
X_train.shape

(714, 2)

In [164]:
len(X_train['Age'].unique()) # определяем количество уникальных значений столбца Age

88

In [165]:
len(X_train['Pclass'].unique()) # определяем количество уникальных значений столбца Pclass

3

In [166]:
Y_train.shape

(714,)

In [167]:
Y_train.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [168]:
X_test.head()

,Age,Pclass
0,22.0,3
1,38.0,1
2,26.0,3
3,35.0,1
4,35.0,3


In [169]:
X_test.shape

(714, 2)

In [170]:
len(X_test['Age'].unique()) # определяем количество уникальных значений столбца Age

88

Применим **one-hot кодирование** для получения данных в виде значений "0" и "1".

In [171]:
X_train_cat = np_utils.to_categorical(X_train, len(X_train['Age'].unique())) # предобрабатываем датасет со значениями "labels" с помощью метода "np_utils" из библиотеки "keras" модуля "utils"; применяем технику "one-hot encoding"; len(X_train['Age'].unique - это количество классов, которые мы задаём в данном датасете
X_train_cat

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)

In [172]:
X_train_cat.shape

(714, 2, 88)

In [173]:
X_train_flat = X_train_cat.reshape(X_train_cat.shape[0], X_train_cat.shape[1] * X_train_cat.shape[2]) # преобразовываем данные 'X_train' в двумерный массив, чтобы нейронная сеть смогла принять входные данные, расскладывая полученное значение 'X_train_cat' на отдельные значения

In [174]:
X_train_flat.shape

(714, 176)

In [175]:
Y_train_cat = np_utils.to_categorical(Y_train, 2)

In [176]:
Y_train_cat.shape

(714, 2)

In [177]:
Y_train_cat

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [178]:
X_test_cat = np_utils.to_categorical(X_test, len(X_test['Age'].unique())) # предобрабатываем датасет со значениями "labels"
X_test_cat

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)

In [179]:
X_test_cat.shape

(714, 2, 88)

In [180]:
X_test_flat = X_test_cat.reshape(X_test_cat.shape[0], X_test_cat.shape[1] * X_test_cat.shape[2]) # преобразовываем данные 'X_test' в двумерный массив

In [181]:
X_test_flat.shape

(714, 176)

In [226]:
X_test_flat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# Создание модели нейронной сети

In [182]:
model = Sequential()

model.add(Dense(512, input_shape=(176, ), activation='relu', name="dense_1")) # добавляем ранее импортированный модуль Dense из библиотеки "keras" в нашу нейронную сеть (модель) - это будет первый скрытый слой нашей модели; задаём количество нейронов - 512; передаём параметр "input_shape" - размерность данных, на которых будет обучаться нейросеть (значение X_train_flat.shape[1]); добавляем активацию "relu"
model.add(Dense(512, activation='relu', name="dense_2")) # добавляем ещё один скрытый слой
model.add(Dropout(0.3))
model.add(Dense(2, activation='softmax', name="output")) # 2 - количество классов (0 и 1 - выжил пассажир или нет); добавляем активацию "softmax", чтобы все вероятности суммировались в единиицу ("1")

In [183]:
model.summary() # выводим описание нашей нейросети

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 512)               90624     
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                                 
 dropout_14 (Dropout)        (None, 512)               0         
                                                                 
 output (Dense)              (None, 2)                 1026      
                                                                 
Total params: 354,306
Trainable params: 354,306
Non-trainable params: 0
_________________________________________________________________


In [184]:
model.compile( # компилируем модель
    loss='categorical_crossentropy', # задаём функцию ошибки (у нас задача классификации)
    optimizer='adam', # оптимизатор (как модель будет обучаться, изменять веса)
    metrics=['accuracy']) # метрики (точность "accuracy")

In [185]:
model.fit(X_train_flat, Y_train_cat, epochs=10, batch_size=128, verbose=1) # обучение модели на тренировочных данных; вызываем метод "fit" для объекта "model"; передаём параметры, на которых будет обучаться модель: X_train_flat (featers), Y_train_cat (labels), epochs (количество итераций через полный набор данных), batch_size (количество экземпляров, которые за раз проходит модель), verbose (определяет то, что будет печатать модель на выходе - progress bar)

Epoch 1/10
6/6 [==============================] - 1s 12ms/step - loss: 0.6601 - accuracy: 0.5980
Epoch 2/10
6/6 [==============================] - 0s 17ms/step - loss: 0.6047 - accuracy: 0.6877
Epoch 3/10
6/6 [==============================] - 0s 11ms/step - loss: 0.5871 - accuracy: 0.6919
Epoch 4/10
6/6 [==============================] - 0s 14ms/step - loss: 0.5726 - accuracy: 0.7129
Epoch 5/10
6/6 [==============================] - 0s 17ms/step - loss: 0.5538 - accuracy: 0.7171
Epoch 6/10
6/6 [==============================] - 0s 13ms/step - loss: 0.5412 - accuracy: 0.7297
Epoch 7/10
6/6 [==============================] - 0s 14ms/step - loss: 0.5247 - accuracy: 0.7451
Epoch 8/10
6/6 [==============================] - 0s 14ms/step - loss: 0.5056 - accuracy: 0.7647
Epoch 9/10
6/6 [==============================] - 0s 14ms/step - loss: 0.4915 - accuracy: 0.7661
Epoch 10/10
6/6 [==============================] - 0s 14ms/step - loss: 0.4850 - accuracy: 0.7745


In [263]:
Y_pred = model.predict(X_test_flat) # делаем предсказание
Y_pred = np.array(Y_pred)
Y_pred = Y_pred.flatten()

23/23 [==============================] - 0s 3ms/step


In [264]:
Y_pred

array([0.7262362 , 0.27376378, 0.2968802 , ..., 0.8520298 , 0.56144804,
       0.43855196], dtype=float32)

In [265]:
Y_pred[Y_pred<0.5] = 0
Y_pred[Y_pred>0] = 1
Y_pred = Y_pred.astype(int)

In [266]:
Y_pred

array([1, 0, 0, ..., 1, 1, 0])

In [270]:
Y_new = Y_pred[:714]
Y_new.shape

(714,)

In [271]:
X_test["Survived"] = Y_new
X_test

,Age,Pclass,Survived
0,22.0,3,1
1,38.0,1,0
2,26.0,3,0
3,35.0,1,1
4,35.0,3,1
...,...,...,...
885,39.0,3,0
886,27.0,2,1
887,19.0,1,0
889,26.0,1,0


In [272]:
X_test.to_csv("result_2.csv")